# JourneyPatternID is a combination of Variant and Direction

## Plan
## Merge on southbound

### from routes_all(headsigns).csv derrive SOUTHBOUND (LineID,JourneyPatternID,Headsign)
#### Step1:
Append County to headsigns
#### Step2:
Derrive Southbound - for a given LineID there are two headsigns, substract the corresponding latitude for each headsign from the other.

### From Timetable - Derive Southbound Column
#### Step4:
derive to_lat
#### Step5:
derive from_lat
#### Step6:
subtract to get southbound


In [2]:
import pandas as pd
import math
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
%matplotlib inline

### Constants

In [3]:
lines =['102', '104', '111', '114', '116', '118', '11', '120', '122', '123',
       '130', '13', '140', '142', '145', '14', '150', '151', '15A', '15B',
       '15', '161', '16', '17A', '17', '184', '185', '18', '1', '220',
       '236', '238', '239', '25A', '25B', '25X', '25', '26', '270', '27A',
       '27B', '27X', '27', '29A', '31B', '31', '32', '32X', '33A', '33B',
       '33X', '33', '37', '38A', '38', '39A', '39', '40B', '40D', '40',
       '41B', '41C', '41X', '41', '42', '43', '44B', '44', '45A', '46A',
       '46E', '47', '49', '4', '51D', '51X', '53', '54A', '56A', '59',
       '61', '63', '65B', '65', '66A', '66X', '66', '67', '68', '69X',
       '69', '70', '747', '75', '76A', '76', '77A', '79', '79A', '7B',
       '7D', '7', '83', '84', '84A', '84X', '8', '9']

service_types = ["weekday","weekday","weekday","weekday","weekday","saturday","sunday"]


# Map Timetable to SOUTHBOUND

In [54]:
#Existing Mapping
timetable_mapping = pd.read_csv('bus_data/static_data/td_map.csv')
timetable_mapping = timetable_mapping.drop('Unnamed: 0', 1)

In [55]:
timetable_mapping.head()

,fullname,LineID,southbound
0,102_saturday_From Dublin Airport Towards Sutto...,102,True
1,102_saturday_From Sutton Station Towards Dubli...,102,False
2,102_sunday_From Dublin Airport Towards Sutton ...,102,True
3,102_sunday_From Sutton Station Towards Dublin ...,102,False
4,102_weekday_From Dublin Airport Towards Sutton...,102,True


In [56]:
timetable_mapping.LineID.unique()

array(['102', '104', '111', '114', '116', '118', '11', '120', '122', '123',
       '130', '13', '140', '142', '145', '14', '150', '151', '15A', '15B',
       '15', '161', '16', '17A', '17', '184', '185', '18', '1', '220',
       '236', '238', '239', '25A', '25B', '25X', '25', '26', '270', '27A',
       '27B', '27X', '27', '29A', '31B', '31', '32B', '32', '32A', '32X',
       '33A', '33B', '33D', '33X', '33', '37', '38A', '38', '39A', '39',
       '40B', '40D', '40', '41B', '41C', '41X', '41', '42', '43', '44B',
       '44', '45A', '46A', '46E', '47', '49', '4', '51D', '51X', '53',
       '54A', '56A', '59', '61', '63', '65B', '65', '66A', '66X', '66',
       '67', '68', '69X', '69', '70', '747', '75', '76A', '76', '77A',
       '79', '79A', '7B', '7D', '7', '83', '84', '84A', '84X', '8', '90',
       '9', nan], dtype=object)

In [57]:
needed = []
for i in lines:
    if i not in timetable_mapping.LineID.unique():
        needed.append(i)
        
needed

[]

In [62]:
#to drop
for i in timetable_mapping.LineID.unique():
    if i not in lines:
        print(i)

In [61]:
timetable_mapping = timetable_mapping[~timetable_mapping['LineID'].isin(['32B','32A','33D','90'])]
timetable_mapping.dropna(axis=0, subset=['LineID'], inplace=True)

In [63]:
timetable_mapping.columns

Index(['fullname', 'LineID', 'southbound'], dtype='object')

In [64]:
timetable_mapping.to_csv('timetable_mapping.csv')

# MERGE TIMETABLES ON 'fullname' TO TIMETABLE MAPPING

# MAP HEADSIGNS TO SOUTHBOUND

In [65]:
heads = pd.read_csv('bus_data/static_data/routes_all(headsigns).csv')

In [68]:
heads.head()

,LineID,JourneyPatternID,Headsign,Towards
0,1,1,Strand Road,"Strand Road, Dublin"
1,1,1001,Shanard Road,"Shanard Road, Dublin"
2,102,1,Airport,"Airport, Dublin"
3,102,1001,Sutton,"Sutton, Dublin"
4,104,1,Shanard Road,"Shanard Road, Dublin"


In [67]:
#Monday is 0, Sunday is 6
def append_dublin(x):
    return x+(', Dublin')
    
heads['Towards'] = heads['Headsign'].apply(append_dublin)

In [20]:
# #Monday is 0, Sunday is 6
# def get_from(x):
#     return x+(', Dublin')

# names[names.JourneyPattern==100]

# if heads.JourneyPatternID == 1
#     heads['From'] = heads.apply(get_from)

SyntaxError: invalid syntax (<ipython-input-20-49049cdecb19>, line 7)

In [24]:
#Monday is 0, Sunday is 6
def get_line(x):
    return x.split("_")[0].upper()
    
names['LineID'] = names['fullname'].apply(get_line)

In [25]:
#Monday is 0, Sunday is 6
def get_direction(x):
    return x.split("_")[-1].strip('.csv')
    
names['direction'] = names['fullname'].apply(get_direction)

In [26]:
def remove_brackets(x):
    return x.split("(")[0]
names['towards'] = names['towards'].apply(remove_brackets)
names['from'] = names['from'].apply(remove_brackets)

In [27]:
from geopy.geocoders import Nominatim

def get_lat(a):
    geolocator = Nominatim()
    location = Nominatim().geocode(a)
    return(location.latitude)

In [92]:
for i in range(len(names['from'].values)):
    g = names['from'].values[i]
    print(g)
    print(get_lat(g))
    print()

Dublin Airport, Dublin
53.4286802

Sutton Station, Dublin
53.3905198

Clontarf Rd. 
53.3629221

Santry 
53.3944773

Dun Laoghaire, Dublin


GeocoderTimedOut: Service timed out

In [74]:
names['from_lat'] = names['from'].apply(get_lat)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [65]:
names['to_lat'] = names['towards'].apply(get_lat)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [66]:
names

,fullname,from,towards,LineID,direction
0,102_saturday_From Dublin Airport Towards Sutto...,"Dublin Airport, Dublin","Sutton Station, Dublin",102,From Dublin Airport Towards Sutton Station
1,102_saturday_From Sutton Station Towards Dubli...,"Sutton Station, Dublin","Dublin Airport, Dublin",102,From Sutton Station Towards Dublin Airport
2,102_sunday_From Dublin Airport Towards Sutton ...,"Dublin Airport, Dublin","Sutton Station, Dublin",102,From Dublin Airport Towards Sutton Station
3,102_sunday_From Sutton Station Towards Dublin ...,"Sutton Station, Dublin","Dublin Airport, Dublin",102,From Sutton Station Towards Dublin Airport
4,102_weekday_From Dublin Airport Towards Sutton...,"Dublin Airport, Dublin","Sutton Station, Dublin",102,From Dublin Airport Towards Sutton Station
5,102_weekday_From Sutton Station Towards Dublin...,"Sutton Station, Dublin","Dublin Airport, Dublin",102,From Sutton Station Towards Dublin Airport
6,104_saturday_From Clontarf Rd. (Conquer Hill) ...,"Clontarf Rd. (Conquer Hill), Dublin","Santry (Shanard Rd.), Dublin",104,From Clontarf Rd. (Conquer Hill) Towards Santr...
7,104_saturday_From Santry (Shanard Rd.) Towards...,"Santry (Shanard Rd.), Dublin","Clontarf Rd. (Conquer Hill), Dublin",104,From Santry (Shanard Rd.) Towards Clontarf Rd....
8,104_sunday_From Clontarf Rd. (Conquer Hill) To...,"Clontarf Rd. (Conquer Hill), Dublin","Santry (Shanard Rd.), Dublin",104,From Clontarf Rd. (Conquer Hill) Towards Santr...
9,104_sunday_From Santry (Shanard Rd.) Towards C...,"Santry (Shanard Rd.), Dublin","Clontarf Rd. (Conquer Hill), Dublin",104,From Santry (Shanard Rd.) Towards Clontarf Rd....


In [62]:
names.head()

,fullname,from,towards,LineID,direction
0,102_saturday_From Dublin Airport Towards Sutto...,"Dublin Airport, Dublin","Sutton Station, Dublin",102,From Dublin Airport Towards Sutton Station
1,102_saturday_From Sutton Station Towards Dubli...,"Sutton Station, Dublin","Dublin Airport, Dublin",102,From Sutton Station Towards Dublin Airport
2,102_sunday_From Dublin Airport Towards Sutton ...,"Dublin Airport, Dublin","Sutton Station, Dublin",102,From Dublin Airport Towards Sutton Station
3,102_sunday_From Sutton Station Towards Dublin ...,"Sutton Station, Dublin","Dublin Airport, Dublin",102,From Sutton Station Towards Dublin Airport
4,102_weekday_From Dublin Airport Towards Sutton...,"Dublin Airport, Dublin","Sutton Station, Dublin",102,From Dublin Airport Towards Sutton Station


In [49]:
names.LineID.unique()

array(['102', '104', '111', '114', '116', '118', '11', '120', '122', '123',
       '130', '13', '140', '142', '145', '14', '150', '151', '15A', '15B',
       '15', '161', '16', '17A', '17', '184', '185', '18', '1', '220',
       '236', '238', '239', '25A', '25B', '25', '25X', '26', '270', '27A',
       '27B', '27', '27X', '29A', '31B', '31', '32', '32B', '32A', '32X',
       '33A', '33B', '33D', '33', '33X', '37', '38A', '38', '39A', '39',
       '40B', '40D', '40', '41B', '41C', '41X', '41', '42', '43', '44B',
       '44', '45A', '46A', '46E', '47', '49', '4', '51D', '51X', '53',
       '54A', '56A', '59', '61', '63', '65B', '65', '66A', '66', '66X',
       '67', '68', '69', '69X', '70', '747', '75', '76A', '76', '77A',
       '79', '79A', '7B', '7D', '7', '83', '84', '84A', '84X', '8', '90',
       '9'], dtype=object)

In [ ]:
#remove lines in timetable not in our data
new_df = names[(names.LineID != '32B')&(names.LineID != '32A')&(names.LineID != '33D')&(names.LineID != '90')]

In [59]:
for i in lines:
    if i not in names.LineID.unique():
        print(i)

In [60]:
for i in new_df.LineID.unique():
    if i not in lines:
        print(i)

## Static Variables

In [3]:
# line = lines[1]
line = "16"
date = timeframes[2]
day = pd.to_datetime(date)
service_type = service_types[day.dayofweek]

## Import Data For given Line

In [4]:
linedata = pd.read_csv("bus_data/line_data/"+line+".csv")
linedata.columns = ["Timestamp", "LineID", "JourneyPatternID", "TimeFrame", 
              "VehicleJourneyID", "Lon", "Lat", "VehicleID", "StopID", 
              "AtStop", "HumanTime", "Day", "Hour", "Runtime"]

# Map first digit of JourneyPatternID to a direction

In [5]:
#break by TimeFrame
#break by JourneyPatternID
#break by VehicleJourneyID
#Check directionality, subtract first lat and last lat

journey_pattern_dict = {}

time_frame_groups = linedata.groupby('TimeFrame')
for time_frame_label, time_frame_df in time_frame_groups:

    journey_pattern_groups = time_frame_df.groupby('JourneyPatternID')

    for journey_pattern_label, journey_pattern_df in journey_pattern_groups:

        vehicle_journey_groups = journey_pattern_df.groupby('VehicleJourneyID')

        for vehicle_journey_label, vehicle_journey_df in vehicle_journey_groups:
            #get first occurence of vehicleJourneyID
            start = vehicle_journey_df.Lat.iloc[0]
            #get last occurence of vehicleJourneyID
            end = vehicle_journey_df.Lat.iloc[-1]

            direction = (start - end)
            #sum should reveal the most common directions
            if direction > 0:
                if not journey_pattern_label in journey_pattern_dict:
                    journey_pattern_dict[journey_pattern_label] = [0,1]
                else:
                    journey_pattern_dict[journey_pattern_label][1] += 1
#                 journey_pattern_dict[journey_pattern_label] = 1
            else:
                if not journey_pattern_label in journey_pattern_dict:
                    journey_pattern_dict[journey_pattern_label] = [1,0]
                else:
                    journey_pattern_dict[journey_pattern_label][0] += 1
#                 journey_pattern_dict[journey_pattern_label] = 0
#             journey_pattern_dict[journey_pattern_label] += (1 if (direction > 0) else 0)


                

In [6]:
journey_pattern_dict

{1: [83, 3327], 2: [3, 327], 3: [1, 40], 4: [0, 159], 1001: [6613, 13]}

In [7]:
type(journey_pattern_dict)
# journey_pattern_dict.values()
for key, value in journey_pattern_dict.items():
    print(key,"is southbound", str(value[0]>value[1]))

1 is southbound False
2 is southbound False
3 is southbound False
4 is southbound False
1001 is southbound True


In [8]:
#Create Dataframe with direction counts and JourneyPatternIDs
df = pd.DataFrame.from_dict(journey_pattern_dict).T
df.index.name = 'JourneyPatternID'
df.columns = ['northbound','southbound']

In [9]:
df.northbound.sum()+ df.southbound.sum()

10566

In [10]:
linedata.shape[0] - (df.northbound.sum()+ df.southbound.sum())

218552

In [11]:
#add southbound into original linedata df based on first digit of journey pattern id
# linedata["Southbound"] = linedata.JourneyPatternID>1000
linedata["Southbound"] = linedata.JourneyPatternID>1000

In [12]:
linedata.head()

,Timestamp,LineID,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,VehicleID,StopID,AtStop,HumanTime,Day,Hour,Runtime,Southbound
0,1352183125000000,16,2,2012-11-06,5705,-6.242439,53.386570,33633,1641,1,2012-11-06 06:25:25,1,6,0,False
1,1352183146000000,16,1001,2012-11-06,5521,-6.282335,53.292206,33428,1328,1,2012-11-06 06:25:46,1,6,43,True
2,1352183205000000,16,2,2012-11-06,5705,-6.245068,53.381519,33633,213,1,2012-11-06 06:26:45,1,6,80,False
3,1352183323000000,16,1001,2012-11-06,5521,-6.284559,53.298477,33428,1331,1,2012-11-06 06:28:43,1,6,220,True
4,1352183346000000,16,2,2012-11-06,5705,-6.250914,53.375160,33633,119,1,2012-11-06 06:29:06,1,6,221,False


# Tally JourneyPatternIDs based on first digit of journey pattern ID

In [13]:
#Split JourneyPatternID based on leading 1 or 0
def jp_directions(value_counts):
    oneway = 0
    another = 0

    for index in value_counts.index:
        if (index<1000):
            oneway += value_counts[index]
        else:
            another += value_counts[index]

    return(oneway,another)

In [14]:
#proof by approximate equality that first digit in JourneyPatternID corresponds to Direction
a,b = jp_directions(linedata.JourneyPatternID.value_counts())
print(a, b)

112850 116268


In [29]:
linedata['HumanTime'] = pd.to_datetime(linedata['HumanTime'])

#### number of journey pattern ids should equal number of directions + variations on each direction

In [16]:
#Monday is 0, Sunday is 6
def get_day(x):
    if x.weekday() == 5:
        return "saturday"
    elif x.weekday() == 6:
        return "sunday"
    else:
        return "weekday"
    
linedata["day"] = linedata['HumanTime'].apply(get_day)  

In [17]:
# #service_type defined above @ static variables
# #df of all busses of that line with that service type
# service = linedata.loc[linedata['day'] == service_type]

# print("service_type JourneyPatternIDs:"+service_type)
# service.JourneyPatternID.value_counts()

## Find Time regularity in infrequent JourneyPatternIDs

In [18]:
#break by TimeFrame
#break by JourneyPatternID
#break by VehicleJourneyID
#Check departure, subtract first lat and last lat

journey_pattern_time_dict = {}

time_frame_groups = linedata.groupby('TimeFrame')
for time_frame_label, time_frame_df in time_frame_groups:

    journey_pattern_groups = time_frame_df.groupby('JourneyPatternID')

    for journey_pattern_label, journey_pattern_df in journey_pattern_groups:

        vehicle_journey_groups = journey_pattern_df.groupby('VehicleJourneyID')

        for vehicle_journey_label, vehicle_journey_df in vehicle_journey_groups:
            #get first occurence of vehicleJourneyID
            departure = vehicle_journey_df.HumanTime.iloc[0]
            
            if not journey_pattern_label in journey_pattern_time_dict:
                journey_pattern_time_dict[journey_pattern_label] = [departure]
            else:
                 journey_pattern_time_dict[journey_pattern_label].append(departure)
    

## Import Timetable

In [43]:
timetable = pd.read_csv("bus_data/static_data/master_timetable.csv")

In [44]:
#reassign columns to remove leading whitespace from variant column and name departure index
timetable.columns = ['departure_index', 'variant', 'service', 'departure', 'direction', 'line']

In [45]:
# schedule = timetable.loc[(((timetable['line'] == line)&(timetable['direction'] == directions[0]))&(timetable['service'] == service))]
# schedule.reset_index(inplace=True)

schedule = timetable.loc[(timetable['line'] == line)&(timetable['service'] == service_type)]
schedule.reset_index(inplace=True)

In [46]:
print(schedule.direction.value_counts()[0])
print(schedule.direction.value_counts()[1])
# print("number of northbound trips: "+str(num_nbound))
# print("number of southound trips: "+str(num_sbound))

83
80


In [86]:
#make line letter uppercase
timetable.line = [x.upper() for x in timetable.line]
# timetable.line.value_counts()

In [47]:
# Derive Timetable
time_frame_groups = linedata.groupby('TimeFrame')

# for time_frame_label, time_frame_df in time_frame_groups:
time_frame_df = time_frame_groups.get_group("2013-01-28")

    
a = time_frame_df.drop_duplicates("VehicleJourneyID", keep="first")

a[a.Southbound==True].shape
   
    
    
    
    
#     vehicle_journey_groups = journey_pattern_df.groupby('VehicleJourneyID')

#     for vehicle_journey_label, vehicle_journey_df in vehicle_journey_groups:
        
        
        

#         journey_pattern_groups = time_frame_df.groupby('JourneyPatternID')

#         for journey_pattern_label, journey_pattern_df in journey_pattern_groups:
            
#             print(journey_pattern_label)
#             print(journey_pattern_df)
            
#             vehicle_journey_groups = journey_pattern_df.groupby('VehicleJourneyID')

#             for vehicle_journey_label, vehicle_journey_df in vehicle_journey_groups:
#                 #get first occurence of vehicleJourneyID
#                 departure = vehicle_journey_df.HumanTime.iloc[0]

#                 if not journey_pattern_label in journey_pattern_time_dict:
#                     journey_pattern_time_dict[journey_pattern_label] = [departure]
#                 else:
#                      journey_pattern_time_dict[journey_pattern_label].append(departure)

(130, 16)

## Is There a Match?

In [ ]:
schedule.variant.value_counts()

## Does the first digit of the JourneyPatternID accord with the derived southbound feature?